Copyright 2020 Montvieux Ltd

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [2]:
import PIL.Image
from IPython.display import display,clear_output,HTML
from IPython.display import Image as DisplayImage
import base64
import json
from io import StringIO
import ipywidgets as widgets
import sys
import time
import datetime
import imageio
import numpy as np
import io
import os
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

from plark_game import classes
from gym_plark.envs import plark_env,plark_env_guided_reward,plark_env_top_left


from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv


import helper 

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Save model location

In [3]:

basicdate = str(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))

basepath = '/data/agents/models'
exp_name = 'test_' + basicdate
exp_path = os.path.join(basepath, exp_name)

print(exp_path)

/data/agents/models/test_20230426_095040


# Use the self play training loop

In [4]:
import self_play
basicdate = str(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))

basepath = '/data/agents/models'
exp_name = 'test_' + basicdate
exp_path = os.path.join(basepath, exp_name)

print(exp_path)
# video_path,basewidth,hsize = self_play.run_self_play(exp_name,exp_path,basicdate,
#                     pelican_testing_interval=1000,pelican_max_initial_learning_steps=10000,
#                     panther_testing_interval=1000,panther_max_initial_learning_steps=10000,
#                     self_play_testing_interval=1000,self_play_max_learning_steps_per_agent=10000,self_play_iterations=10,
#                     model_type='dqn',log_to_tb=True                                 
#                 )

video_path,basewidth,hsize = self_play.run_self_play(exp_name,exp_path,basicdate,
                    pelican_testing_interval=100,pelican_max_initial_learning_steps=1,
                    panther_testing_interval=100,panther_max_initial_learning_steps=1,
                    self_play_testing_interval=100,self_play_max_learning_steps_per_agent=1,self_play_iterations=1,
                    model_type='PPO',log_to_tb=False                                 
                )


/data/agents/models/test_20230426_095043


2023-04-26 09:50:45.679800: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12
INFO:gym_plark.envs.plark_env:plark.kwargs :{'driving_agent': 'pelican', 'image_based': True, 'random_panther_start_position': True, 'max_illegal_moves_per_turn': 5, 'render_height': 250, 'render_width': 310}
INFO:gym_plark.envs.plark_env:self.image_based :True
INFO:gym_plark.envs.plark_env:config filepath: /Components/plark-game/plark_game/game_config/10x10/pelican_easy.json
INFO:plark_game.classes.environment:Opening config from:/Components/plark-game/plark_game/game_config/10x10/pelican_easy.json
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pelicanAgent_3_buoys.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/PantherAgentIllegalMove.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pelica

INFO:plark_game.classes.pelicanAgent_load_agent:pelican agent loaded
INFO:plark_game.classes.environment:Game Created
INFO:gym_plark.envs.plark_env:observation space: Height:250, Width:310, Channels:3
INFO:gym_plark.envs.plark_env:Image observations created
INFO:self_play:Training initial panther
INFO:self_play:Beginning training for 1 steps
INFO:self_play:Training for 100 steps


Process ForkServerProcess-1:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/vec_env/subproc_vec_env.py", line 34, in _worker
    cmd, data = remote.recv()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 414, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Process ForkServerProcess-2:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, 

KeyboardInterrupt: 

In [ ]:
# video_path = '/data/agents/models/test_20200325_184254/test_self_play.mp4'
# basewidth = 310
# hsize = 250
video = io.open(video_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" width="'''+str(basewidth)+'''" height="'''+str(hsize)+'''" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

# Make video of previously trained agents 

In [ ]:
# Please update the paths below.
panther_agent_filepath = '/data/agents/models/test_20200409_160900/PPO2_20200409_160900_panther/'
pelican_env = plark_env.PlarkEnv(driving_agent='pelican',panther_agent_filepath=panther_agent_filepath,config_file_path='/Components/plark-game/plark_game/game_config/10x10/balanced.json')
pelican_load_path = '/data/agents/models/test_20200409_160900/PPO2_20200409_160900_pelican/PPO2_20200409_160900_pelican.zip'
pelican_model = PPO2.load(pelican_load_path)


In [ ]:
video_path = '/data/test_video/'
os.makedirs(video_path, exist_ok=True)
video_file_path =  os.path.join(video_path, 'test_self_play.mp4') 
basewidth,hsize = helper.make_video(pelican_model,pelican_env,video_file_path)

In [ ]:
video = io.open(video_file_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" width="'''+str(basewidth)+'''" height="'''+str(hsize)+'''" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))